<a href="https://colab.research.google.com/github/RVarshinee/ResumeRankingSystem/blob/main/ResumeModelTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U spacy
!pip install spacy_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.8/190.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.5 MB/s eta 0:00:00


In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [ ]:
cv_data=json.load(open('/content/drive/MyDrive/dataset/dataset.json','r'))

In [ ]:
!python -m spacy init fill-config /content/drive/MyDrive/config/base_config.cfg /content/drive/MyDrive/config/config.cfg

2023-09-15 15:54:00.942137: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/config/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
def get_spacy_doc(file,data):
  nlp=spacy.blank("en")
  db=DocBin()
  for text,annot  in tqdm(data):
    doc=nlp.make_doc(text)
    annot=annot['entities']
    ents=[]
    entity_indices=[]
    for start,end,label in annot:
      skip_entity=False
      for idx in range(start,end):
        if idx in entity_indices:
          skip_entity=True
          break
      if skip_entity==True:
        continue
      entity_indices=entity_indices+list(range(start,end))
      try:
        span=doc.char_span(start,end,label=label,alignment_mode='strict')
      except:
        continue
      if span is None:
        err_data=str([start,end])+"     "+str(text)+"\n"
        file.write(err_data)
      else:
        ents.append(span)
    try:
      doc.ents=ents
      db.add(doc)
    except:
      pass
  return db

In [ ]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(cv_data,test_size=0.3)

In [ ]:
file=open('/content/drive/MyDrive/model/train_file.txt','w')
db=get_spacy_doc(file,train)
db.to_disk('/content/drive/MyDrive/model/train_data.spacy')
db=get_spacy_doc(file,test)
db.to_disk('/content/drive/MyDrive/model/test_data.spacy')
file.close()

100%|██████████| 305/305 [00:02<00:00, 135.94it/s]


In [ ]:
pip install spacy-transformers

In [ ]:
!python -m spacy train /content/drive/MyDrive/config/config.cfg --output /content/drive/MyDrive/model/output --paths.train  /content/drive/MyDrive/model/train_data.spacy --paths.dev /content/drive/MyDrive/model/test_data.spacy --gpu-id 0

2023-09-15 16:06:51.574345: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: /content/drive/MyDrive/model/output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not i

In [ ]:
nlp=spacy.load('/content/drive/MyDrive/model/output/model-best')

In [ ]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 51.2 MB/s eta 0:00:00


In [ ]:
import sys,fitz
fname='/content/drive/MyDrive/11197262.pdf'
doc=fitz.open(fname)

In [ ]:
text=" "
for page in doc:
  text=text+str(page.get_text())

In [ ]:
doc=nlp(text)
for ent in doc.ents:
  print(ent.text,"    ->>>>>>>>>>    ",ent.label_)

Certificate study in Wetlands
Identification and Delineation
Proficient in Microsoft Office suite     ->>>>>>>>>>     CERTIFICATION
Keystone College Academic Honor Award -Dean's List Spring 2011 and Fall 2008
     ->>>>>>>>>>     COLLEGE NAME
Plant Protection and Quarantine Technician     ->>>>>>>>>>     WORKED AS
Forest Insect Pest Aide I     ->>>>>>>>>>     WORKED AS
Forest Insect Pest Aide     ->>>>>>>>>>     WORKED AS
Trail Steward and Workshop Staff Member     ->>>>>>>>>>     WORKED AS
Flammulated Owl Research Lab Technician     ->>>>>>>>>>     COMPANIES WORKED AT
Wildlife Biology Field Assistant     ->>>>>>>>>>     WORKED AS
Bachelor of Science : Environmental Biology , 2011 Keystone College ï¼​ City , State , US     ->>>>>>>>>>     DEGREE
Keystone College Eco Club     ->>>>>>>>>>     COLLEGE NAME
American Red Cross First Aid and CPR training     ->>>>>>>>>>     CERTIFICATION
FEMA Federal Emergency Management Training
Skills     ->>>>>>>>>>     CERTIFICATION
